In [ ]:
import numpy as np
import tqdm
import requests
import pandas as pd
import os
import numpy as np
import random
import json
from collections import Counter
import re
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import math

import pandas as pd
import numpy as np
import re
from collections import Counter
import math
import aiohttp
import asyncio


In [30]:
class ColumnAnalysis:

    def __init__(self):
        self.entity_type_dict = {
            "PERSON": "NE",
            "NORP": "NE",
            "FAC": "NE",
            "ORG": "NE",
            "GPE": "NE",
            "LOC": "NE",
            "PRODUCT": "NE",
            "EVENT": "NE",
            "WORK_OF_ART": "NE",
            "LAW": "NE",
            "LANGUAGE": "NE",
            "DATE": "LIT",
            "TIME": "LIT",
            "PERCENT": "LIT",
            "MONEY": "LIT",
            "QUANTITY": "LIT",
            "ORDINAL": "LIT",
            "CARDINAL": "LIT",
            "URL": "LIT",
            "DESC": "LIT",
            "TOKEN": "NE",
            "INTEGER": "LIT",
            "FLOAT": "LIT",
            "DATETIME": "LIT",
            "ADDRESS": "LIT",
            "EMAIL": "LIT"
        }

        self.LIT_DATATYPE = {
            "DATE": "DATETIME", 
            "TIME": "STRING", 
            "PERCENT": "STRING", 
            "MONEY": "STRING", 
            "QUANTITY": "STRING", 
            "ORDINAL": "NUMBER", 
            "CARDINAL": "NUMBER", 
            "URL": "STRING",
            "DESC": "STRING",
            "TOKEN": "STRING",
            "INTEGER": "NUMBER",
            "FLOAT": "NUMBER",
            "DATETIME": "DATETIME",
            "ADDRESS": "STRING",
            "EMAIL": "STRING",
            "STRING": "STRING"
        }

        self.NE_DATATYPE = ["PERSON", "NORP", "FAC", "ORG", "GPE", "LOC", "PRODUCT", "EVENT", "WORK_OF_ART", "LAW", "LANGUAGE"]
    
    def most_frequent_element(self, input_list):
        counter = Counter(input_list)
        most_common = counter.most_common(1)
        return most_common[0][0] if most_common else None

    def extract_number_features(self, column):
        try:
            col = pd.to_numeric(column, errors='coerce')
            return {
                'min_value': np.min(col),
                'max_value': np.max(col),
                'mean_value': np.mean(col),
                'std_dev': np.std(col),
                'unique_count': len(set(col))
            }
        except Exception as e:
            print(f"Error extracting number features: {e}")
            return {}

    def extract_named_entity_features(self, column):
        lengths = [len(str(entry)) for entry in column]
        features = {
            'average_length': np.mean(lengths) if lengths else 0,
            'min_length': np.min(lengths) if lengths else 0,
            'max_length': np.max(lengths) if lengths else 0,
            'all_caps': sum(1 for entry in column if str(entry).isupper()),
            'capitalized': sum(1 for entry in column if str(entry).istitle()),
            'hyphens': sum(str(entry).count('-') for entry in column),
            'periods': sum(str(entry).count('.') for entry in column),
            'commas': sum(str(entry).count(',') for entry in column)
        }
        return features

    def extract_string_features(self, column):
        lengths = [len(str(entry)) for entry in column]
        features = {
            'average_length': np.mean(lengths) if lengths else 0,
            'min_length': np.min(lengths) if lengths else 0,
            'max_length': np.max(lengths) if lengths else 0,
            'all_caps': sum(1 for entry in column if str(entry).isupper()),
            'capitalized': sum(1 for entry in column if str(entry).istitle()),
            'alphabetic_chars': sum(char.isalpha() for entry in column for char in str(entry)),
            'digit_chars': sum(char.isdigit() for entry in column for char in str(entry)),
            'special_chars': sum(not char.isalnum() for entry in column for char in str(entry))
        }
        return features

    def extract_datetime_features(self, column):
        dates = pd.to_datetime(column, errors='coerce')
        features = {
            'min_date': dates.min(),
            'max_date': dates.max(),
            'date_range': (dates.max() - dates.min()).days,
            'year_counts': dates.dt.year.value_counts().to_dict(),
            'month_counts': dates.dt.month.value_counts().to_dict()
        }
        return features

    async def fetch_entity(self, session, cell):
        if cell is None or pd.isna(cell):
            return None
        cell = str(cell)
        url = 'https://lamapi.hel.sintef.cloud/lookup/entity-retrieval'
        params = {
            'name': cell,
            'token': 'lamapi_demo_2023',
            'kg': 'wikidata',
            'limit': 10,
            'query': f'{{"query": {{"bool": {{"must": [{{"match": {{"name": {{"query": "{cell}", "boost": 2.0}}}}}}]}}}}}}',
            'sort': [
                f'''{{"popularity": {{"order": "desc"}}}}'''
            ]
        }
        async with session.get(url, params=params) as response:
            if response.status == 200:
                return await response.json()
            return None

    async def classify_columns_async(self, df):
        def combine_scores(j_score, ed_score, w1=0.5, w2=0.5):
            return w1 * j_score + w2 * ed_score

        url_pattern = re.compile(r'^(https?|ftp)://[^\s/$.?#].[^\s]*$', re.IGNORECASE)
        email_pattern = re.compile(r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$', re.IGNORECASE)
        address_pattern = re.compile(r'\d+\s+\w+\s+(?:street|st|avenue|ave|road|rd|boulevard|blvd|lane|ln|drive|dr|court|ct|circle|cir|place|pl)\.?\s*\w*', re.IGNORECASE)
        datetime_pattern = re.compile(
            r'(?:\d{4}-\d{2}-\d{2})'  # YYYY-MM-DD format
            r'|(?:31(?:\/|-|\.)0?[13578]|1[02](?:\/|-|\.)\d{4})'  # 31 days months
            r'|(?:29|30(?:\/|-|\.)0?[1,3-9]|1[0-2](?:\/|-|\.)\d{4})'  # 29/30 days months
            r'|(?:0?[1-9]|[12]\d|3[01])(?:\/|-|\.)'  # Day
            r'(?:0?[1-9]|1[0-2])(?:\/|-|\.)\d{4}'  # Month
            r'|(?:0?[1-9]|1[0-2])/(?:0?[1-9]|[12]\d|3[01])/(?:\d{2})'  # MM/DD/YY format
            r'|(?:0?[1-9]|1[0-2])/(?:0?[1-9]|[12]\d|3[01])/\d{2}'  # MM/DD/YY format
            r'\b\d{2}/(?:0?[1-9]|[12]\d|3[01])/(?:0?[1-9]|1[0-2])\b'  # YY/DD/MM format
            r'|(?:[01]?\d|2[0-3]):[0-5]\d\.[0-5]\d'  # HH:MM.SS format
            r'|(?:[01]?\d|2[0-3]):[0-5]\d'  # HH:MM format
            r'|(?:[0-5]?\d):[0-5]\d(?:\.\d{1,2})?'  # H:MM or H:MM.S format
            r'|(?:2[0-3]|[01]?\d)h[0-5]?\d(?:m[0-5]?\d(?:\.\d{1,2})?s)?',  # HhMMmSSs format
            re.IGNORECASE
        )

        col_type = []
        feature_list = []

        async with aiohttp.ClientSession() as session:
            for col_name, col_data in df.items():
                type = []
                count_cell = 0

                for cell in col_data:
                    label = None
                    is_number = False
    
                    try:
                        if math.isnan(cell):
                            label = "None"
                    except:
                        pass
                        
                    if isinstance(cell, str):
                        if cell == "NaN" or cell == "nan":
                            label = "None"
                        elif re.match(url_pattern, cell):
                            label = "URL"
                        elif re.match(email_pattern, cell):
                            label = "EMAIL"
                        elif re.match(address_pattern, cell):
                            label = "ADDRESS"
                        elif re.match(datetime_pattern, cell):
                            label = "DATETIME"
                    
                    if label is None:
                        try:
                            cell_str = str(cell)
                            if ',' in cell_str or '.' in cell_str or '%' in cell_str or '$' in cell_str:
                                cell_str = cell_str.replace('.', '').replace(',', '').replace('%', '').replace('$', '')
                            if len(cell_str) - len(re.findall(r'\d', cell_str)) < 5 and len(re.findall(r'\d', cell_str)) != 0:
                                is_number = True
                        except:
                            pass
                    
                    if is_number:
                        label = "NUMBER"
                    elif label != "None" and len(cell.split(" ")) >= 15:
                        label = "NOA"
                    elif label != "None" and len(cell.split(" ")) >= 1 and len(cell) <= 4:
                        label = "STRING"
                    
                    if label is not None:
                        type.append(label)
                        break
                    else:
                        if count_cell > 5:
                            type.append("STRING")
                            break
                        else:                
                            tasks = [self.fetch_entity(session, cell) for cell in col_data if cell is not None and count_cell <= 5]
                            responses = await asyncio.gather(*tasks)
                            
                            for cell, data in zip(col_data, responses):
                                if data and len(data) > 0 and data[0]['NERtype'] != None:
                                    if combine_scores(data[0]['jaccard_score'], data[0]['ed_score']) >= 0.7:
                                        type.append(f"NE_{data[0]['NERtype']}")
                                else:
                                    type.append("STRING")
                                count_cell += 1

                most_common_type = self.most_frequent_element(type)
                col_type.append(most_common_type)

                if most_common_type == "NUMBER":
                    features = self.extract_number_features(col_data)
                elif most_common_type in ['NE_PERS', 'NE_LOC', 'NE_ORG', 'NE_OTHERS']:
                    features = self.extract_named_entity_features(col_data)
                elif most_common_type == "STRING":
                    features = self.extract_string_features(col_data)
                elif most_common_type == "DATETIME":
                    features = self.extract_datetime_features(col_data)
                else:
                    features = {}

                features['column_name'] = col_name
                features['column_type'] = most_common_type
                feature_list.append(features)

        return feature_list

    def classify_columns(self, df):
        loop = asyncio.get_event_loop()
        return loop.run_until_complete(self.classify_columns_async(df))


In [ ]:
async def process_table(column_analysis, table_path, train_df, columns):
    df = pd.read_csv(table_path)
    result = column_analysis.classify_columns(df.iloc[1:10])

    for entry in result:
        row = {col: [entry.get(col, None)] for col in columns}
        train_df = pd.concat([train_df, pd.DataFrame(row)], ignore_index=True)

    return train_df

async def main(tables_path):
    column_analysis = ColumnAnalysis()
    columns = [
        'column_name', 'column_type', 'min_value', 'max_value', 'mean_value', 'std_dev', 'unique_count', 'special_values',
        'average_length', 'min_length', 'max_length', 'all_caps', 'capitalized', 'hyphens', 'periods', 'commas', 'common_prefixes', 'common_suffixes',
        'alphabetic_chars', 'digit_chars', 'special_chars', 'min_date', 'max_date', 'date_range', 'year_counts', 'month_counts'
    ]
    train_df = pd.DataFrame(columns=columns)

    table_files = [os.path.join(tables_path, table) for table in os.listdir(tables_path)]

    for table_file in tqdm(table_files):
        train_df = await process_table(column_analysis, table_file, train_df, columns)

    return train_df

if __name__ == "__main__":
    tables_path = "./data/Dataset/Dataset/Round1_T2D/tables/"
    # tables_path = "./data/Dataset/Dataset/Round3_2019/tables/"
    train_df = await (main(tables_path))

In [ ]:
output_csv_path = "./R1_train_df.csv"
train_df.to_csv(output_csv_path, index=False)

In [ ]:
async def process_table(column_analysis, table_path, train_df, columns):
    df = pd.read_csv(table_path)
    result = await column_analysis.classify_columns_async(df.iloc[1:10])

    for entry in result:
        row = {col: [entry.get(col, None)] for col in columns}
        train_df = pd.concat([train_df, pd.DataFrame(row)], ignore_index=True)

    return train_df

async def main(tables_path):
    column_analysis = ColumnAnalysis()
    columns = [
        'column_name', 'column_type', 'min_value', 'max_value', 'mean_value', 'std_dev', 'unique_count', 'special_values',
        'average_length', 'min_length', 'max_length', 'all_caps', 'capitalized', 'hyphens', 'periods', 'commas', 'common_prefixes', 'common_suffixes',
        'alphabetic_chars', 'digit_chars', 'special_chars', 'min_date', 'max_date', 'date_range', 'year_counts', 'month_counts'
    ]
    train_df = pd.DataFrame(columns=columns)

    table_files = [os.path.join(tables_path, table) for table in os.listdir(tables_path)]

    for table_file in tqdm(table_files):
        train_df = await process_table(column_analysis, table_file, train_df, columns)

    return train_df

if __name__ == "__main__":
    #tables_path = "./data/Dataset/Dataset/Round1_T2D/tables/"
    tables_path = "./data/Dataset/Dataset/Round3_2019/tables/"
    train_df = await (main(tables_path))

  0%|          | 0/2161 [00:00<?, ?it/s]/tmp/ipykernel_1309/204691845.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  train_df = pd.concat([train_df, pd.DataFrame(row)], ignore_index=True)
  0%|          | 1/2161 [00:01<46:15,  1.28s/it]/tmp/ipykernel_1309/204691845.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  train_df = pd.concat([train_df, pd.DataFrame(row)], ignore_index=True)
  0%|          | 2/2161 [00:02<42:33,  1.18s/it]/tmp/ipykernel_1309/204691845.py:7: FutureWarning: The beh

CancelledError: 

In [ ]:
output_csv_path = "./R3_train_df.csv"
train_df.to_csv(output_csv_path, index=False)


In [ ]:
async def process_table(column_analysis, table_path, train_df, columns):
    df = pd.read_csv(table_path)
    result = await column_analysis.classify_columns_async(df.iloc[1:10])

    for entry in result:
        row = {col: [entry.get(col, None)] for col in columns}
        train_df = pd.concat([train_df, pd.DataFrame(row)], ignore_index=True)

    return train_df

async def main(tables_path):
    column_analysis = ColumnAnalysis()
    columns = [
        'column_name', 'column_type', 'min_value', 'max_value', 'mean_value', 'std_dev', 'unique_count', 'special_values',
        'average_length', 'min_length', 'max_length', 'all_caps', 'capitalized', 'hyphens', 'periods', 'commas', 'common_prefixes', 'common_suffixes',
        'alphabetic_chars', 'digit_chars', 'special_chars', 'min_date', 'max_date', 'date_range', 'year_counts', 'month_counts'
    ]
    train_df = pd.DataFrame(columns=columns)

    pattern = r'^\.'

    # Create a list of file paths, excluding files that start with a dot
    table_files = [os.path.join(tables_path, table) for table in os.listdir(tables_path) if not re.match(pattern, table)]

    for table_file in tqdm(table_files):
        train_df = await process_table(column_analysis, table_file, train_df, columns)

    return train_df

if __name__ == "__main__":
    #tables_path = "./data/Dataset/Dataset/Round1_T2D/tables/"
    tables_path = "./data/Dataset/Dataset/HardTablesR2/tables/"
    train_df = await (main(tables_path))

In [ ]:
output_csv_path = "./HT2_train_df.csv"
train_df.to_csv(output_csv_path, index=False)


In [ ]:
train_df = pd.read_csv("./R1_train_df.csv")

In [ ]:
train_df[['column_type','average_length', 'min_length', 'max_length', 'all_caps', 'capitalized', 'hyphens',
    'periods', 'commas']][train_df['column_type'].isin(['NE_PERS', 'NE_LOC', 'NE_ORG', 'NE_OTHERS'])]

## Round1_T2D

In [ ]:
####################
# READ THE JSON
#####################

json_file_path = "./data/Round1_T2D_f3_sorted_mentions.json"

# Load the JSON file
with open(json_file_path, "r") as file:
    R1_sorted_mentions = json.load(file)

R1_cea = [item[0]for item in R1_sorted_mentions]

In [ ]:
categories = [
    "Place",
    "PopulatedPlace",
    "City",
    "Country",
    "Region",
    "Mountain",
    "Island",
    "Lake",
    "River",
    "Park",
    "Building",
    "HistoricPlace",
    "Monument",
    "Bridge",
    "Road",
    "Airport",
    "Person",
    "Artist",
    "Athlete",
    "Politician",
    "Scientist",
    "Writer",
    "Actor",
    "Musician",
    "MilitaryPerson",
    "Religious",
    "Royalty",
    "Criminal",
    "Organisation",
    "Company",
    "EducationalInstitution",
    "PoliticalParty",
    "SportsTeam",
    "Non-ProfitOrganisation",
    "GovernmentAgency",
    "ReligiousOrganisation",
    "Band",
    "Library",
    "Museum",
    "Hospital",
    "University",
    "TradeUnion"
]

# Mapping of subtypes to macro classes
mapping = {
    "Place": ["PopulatedPlace", "City", "Country", "Region", "Mountain", "Island", "Lake", "River", "Park", "Building", "HistoricPlace", "Monument", "Bridge", "Road", "Airport"],
    "Person": ["Artist", "Athlete", "Politician", "Scientist", "Writer", "Actor", "Musician", "MilitaryPerson", "Religious", "Royalty", "Criminal"],
    "Organisation": ["Company", "EducationalInstitution", "PoliticalParty", "SportsTeam", "Non-ProfitOrganisation", "GovernmentAgency", "ReligiousOrganisation", "Band"],
    "Institution": ["Library", "Museum", "Hospital", "University", "TradeUnion"]
}

In [ ]:
tables = "./data/Dataset/Dataset/Round1_T2D/tables/"

def count_numbers_in_string(s):
    return len(re.findall(r'\d+', str(s)))

median_lengths = []
median_token_counts = []
average_numeric_counts = []
target = []
columns = []

# Iterate through each table
for table in tqdm(os.listdir(tables)):
    table_file = os.path.join(tables, table)
    table_name = table.split(".")[0]
    df = pd.read_csv(table_file)
    
    for col in df.columns:
        column = df[col].astype(str)
        
        # Calculate median length for the current column
        median_length = column.apply(len).median()
        median_lengths.append(median_length)
        
        # Calculate median token count for the current column
        median_token_count = column.apply(lambda x: len(x.split())).median()
        median_token_counts.append(median_token_count)
        
        # Calculate average count of numeric values in the current column
        total_numeric_count = column.apply(count_numbers_in_string).sum()
        average_numeric_count = total_numeric_count / len(df) if len(df) > 0 else 0
        average_numeric_counts.append(average_numeric_count)

        
        # Check for NE flag
        NE_flag = column.isin(R1_cea).any()
        if NE_flag:
            joined_cells = column.str.cat(sep='-')        
            doc = nlp(joined_cells)
            entities = {"ORG": [], "PERS": [], "LOC": [], "OTHERS": []}

            # Extract entities and classify them
            for ent in doc.ents:
                if ent.label_ == "ORG":
                    entities["ORG"].append(ent.text)
                elif ent.label_ == "PERSON":
                    entities["PERS"].append(ent.text)
                elif ent.label_ == "GPE" or ent.label_ == "FAC":  # GPE (Geopolitical Entity)
                    entities["LOC"].append(ent.text)
                else:
                    entities["OTHERS"].append(ent.text)
            
            # Find the key of the longest entities list
            longest_key = max(entities, key=lambda k: len(entities[k]))
            print(f"{joined_cells} --> The key of the longest list is '{longest_key}'")
                    
            target.append("NE")
        elif median_length - average_numeric_count < 2:
            target.append("lit")
        else:
            target.append("None")
    
    columns.extend(df.columns.tolist())

## Round3_2019

In [ ]:
####################
# READ THE JSON
#####################

json_file_path = "./data/Round3_2019_sorted_mentions.json"

# Load the JSON file
with open(json_file_path, "r") as file:
    R3_sorted_mentions = json.load(file)

R3_cea = [item[0]for item in R3_sorted_mentions]

In [ ]:
tables = "./data/Dataset/Dataset/Round3_2019/tables/"

def count_numbers_in_string(s):
    return len(re.findall(r'\d+', str(s)))

median_lengths = []
median_token_counts = []
average_numeric_counts = []
target = []
columns = []

# Iterate through each table
for table in tqdm(os.listdir(tables)):
    table_file = os.path.join(tables, table)
    table_name = table.split(".")[0]
    df = pd.read_csv(table_file)
    
    for col in df.columns:
        column = df[col].astype(str)
        
        # Calculate median length for the current column
        median_length = column.apply(len).median()
        median_lengths.append(median_length)
        
        # Calculate median token count for the current column
        median_token_count = column.apply(lambda x: len(x.split())).median()
        median_token_counts.append(median_token_count)
        
        # Calculate average count of numeric values in the current column
        total_numeric_count = column.apply(count_numbers_in_string).sum()
        average_numeric_count = total_numeric_count / len(df) if len(df) > 0 else 0
        average_numeric_counts.append(average_numeric_count)
        
        # Check for NE flag
        NE_flag = column.isin(R3_cea).any()
        if NE_flag:
            target.append("NE")
        elif median_length - average_numeric_count < 2:
            target.append("lit")
        else:
            target.append("None")
    
    columns.extend(df.columns.tolist())

## 2T_Round4

In [ ]:
####################
# READ THE JSON
#####################

json_file_path = "./data/2T_Round4_sorted_mentions.json"

# Load the JSON file
with open(json_file_path, "r") as file:
    R4_sorted_mentions = json.load(file)

R4_2T_cea = [item[0]for item in R4_sorted_mentions]

In [ ]:
tables = "./data/Dataset/Dataset/2T_Round4/tables/"

def count_numbers_in_string(s):
    return len(re.findall(r'\d+', str(s)))

median_lengths = []
median_token_counts = []
average_numeric_counts = []
target = []
columns = []

# Iterate through each table
for table in tqdm(os.listdir(tables)):
    table_file = os.path.join(tables, table)
    table_name = table.split(".")[0]
    df = pd.read_csv(table_file)
    
    for col in df.columns:
        column = df[col].astype(str)
        
        # Calculate median length for the current column
        median_length = column.apply(len).median()
        median_lengths.append(median_length)
        
        # Calculate median token count for the current column
        median_token_count = column.apply(lambda x: len(x.split())).median()
        median_token_counts.append(median_token_count)
        
        # Calculate average count of numeric values in the current column
        total_numeric_count = column.apply(count_numbers_in_string).sum()
        average_numeric_count = total_numeric_count / len(df) if len(df) > 0 else 0
        average_numeric_counts.append(average_numeric_count)
        
        # Check for NE flag
        NE_flag = column.isin(R4_2T_cea).any()
        if NE_flag:
            target.append("NE")
        elif median_length - average_numeric_count < 2:
            target.append("lit")
        else:
            target.append("None")
    
    columns.extend(df.columns.tolist())

## Round4

In [ ]:
####################
# READ THE JSON
#####################

json_file_path = "./data/Round4_sorted_mentions.json"

# Load the JSON file
with open(json_file_path, "r") as file:
    R4_sorted_mentions = json.load(file)

R4_cea = [item[0]for item in R4_sorted_mentions]

In [ ]:
tables = "./data/Dataset/Dataset/Round4_2020/tables/"

def count_numbers_in_string(s):
    return len(re.findall(r'\d+', str(s)))

median_lengths = []
median_token_counts = []
average_numeric_counts = []
target = []
columns = []

# Iterate through each table
for table in tqdm(os.listdir(tables)):
    table_file = os.path.join(tables, table)
    table_name = table.split(".")[0]
    df = pd.read_csv(table_file)
    
    for col in df.columns:
        column = df[col].astype(str)
        
        # Calculate median length for the current column
        median_length = column.apply(len).median()
        median_lengths.append(median_length)
        
        # Calculate median token count for the current column
        median_token_count = column.apply(lambda x: len(x.split())).median()
        median_token_counts.append(median_token_count)
        
        # Calculate average count of numeric values in the current column
        total_numeric_count = column.apply(count_numbers_in_string).sum()
        average_numeric_count = total_numeric_count / len(df) if len(df) > 0 else 0
        average_numeric_counts.append(average_numeric_count)
        
        # Check for NE flag
        NE_flag = column.isin(R4_cea).any()
        if NE_flag:
            target.append("NE")
        elif median_length - average_numeric_count < 2:
            target.append("lit")
        else:
            target.append("None")
    
    columns.extend(df.columns.tolist())

## DF creation

In [ ]:
# Create the DataFrame
df_def = pd.DataFrame({
    'column names': columns,
    'median_lengths': median_lengths,
    'median_token_counts': median_token_counts,
    'average_numeric_counts': average_numeric_counts,
    'target': target
})

In [ ]:
df_def.to_csv('./data/NE_lit_dataset.csv', index=False)

In [ ]:
###################################
#   READ DIRECTLY THE DATASET HERE
###################################

df = pd.read_csv('./data/NE_lit_dataset.csv')
filtered_df = df[df['target'].isin(['lit', 'NE'])]

# Displaying the filtered DataFrame
df[:7]

In [ ]:
target_counts = df['target'].value_counts()

# Extract counts for specific values
ne_count = target_counts.get("NE", 0)
lit_count = target_counts.get("lit", 0)
none_count = df.shape[0] - (ne_count+lit_count)

print(f"Count of 'NE': {ne_count}")
print(f"Count of 'lit': {lit_count}")
print(f"Count of 'NaN': {none_count}")

## Model training

In [ ]:
df1 = pd.read_csv("./R1_train_df.csv")
df2 = pd.read_csv("./R3_train_df.csv")
df3 = pd.read_csv("./HT2_train_df.csv")

result = pd.concat([df1, df2, df3], axis=0)
result.drop(['date_range', 'year_counts', 'month_counts'], axis=1, inplace=True)

In [ ]:
# Convert the target variable to numeric
label_encoder = LabelEncoder()
result['column_type'] = label_encoder.fit_transform(result['column_type'])

result['min_date'] = pd.to_datetime(result['min_date'], format='%Y-%m-%d %H:%M:%S')
result['max_date'] = pd.to_datetime(result['max_date'], format='%Y-%m-%d %H:%M:%S')
result['min_month'] = result['min_date'].dt.month
result['min_year'] = result['min_date'].dt.year
result['max_month'] = result['max_date'].dt.month
result['max_year'] = result['max_date'].dt.year



In [ ]:
result.iloc[:, 2:21] = result.iloc[:, 2:21].fillna(-1) 
result.iloc[:, 23:27] = result.iloc[:, 23:27].fillna(0)  # fill the ['min_month', 'min_year', 'max_month', 'max_year'] 

X = result.drop(['max_date', 'min_date', 'column_name', 'column_type'], axis=1)  # Drop the target column from features
y = result['column_type'].values
# One-hot encode the target variable for multiclass classification
y = to_categorical(y)

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Define the model
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(y_train.shape[1], activation='softmax')  # Output layer with sigmoid activation for binary classification
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy*100:.2f}%')

## New predictions (should be correct)

In [52]:
datetime_strings = [
    "2023-02-15 08:30:00",
    "2023-03-20 18:45:00",
    "2023-04-10 10:00:00",
    "2023-05-05 14:20:00"
]
df = pd.DataFrame({'desc': ["maggiore di tre anni" for i in range(0, 10)]})
#df = pd.DataFrame({'desc': datetime_strings})
column_analysis = ColumnAnalysis()
df_feat = await column_analysis.classify_columns_async(df)
columns = [
        'min_value', 'max_value', 'mean_value', 'std_dev', 'unique_count', 'special_values',
        'average_length', 'min_length', 'max_length', 'all_caps', 'capitalized', 'hyphens', 'periods', 'commas', 'common_prefixes', 'common_suffixes',
        'alphabetic_chars', 'digit_chars', 'special_chars', 'min_date', 'max_date', 'date_range', 'year_counts', 'month_counts'
    ]


test_df = pd.DataFrame()

for entry in df_feat:
    row = {col: [entry.get(col, None)] for col in columns}
    test_df = pd.DataFrame(row)


In [53]:
# Assuming 'test_df' is your DataFrame name
test_df['min_date'] = pd.to_datetime(test_df['min_date'], format='%Y-%m-%d %H:%M:%S')
test_df['max_date'] = pd.to_datetime(test_df['max_date'], format='%Y-%m-%d %H:%M:%S')
test_df['min_month'] = test_df['min_date'].dt.month
test_df['min_year'] = test_df['min_date'].dt.year
test_df['max_month'] = test_df['max_date'].dt.month
test_df['max_year'] = test_df['max_date'].dt.year



In [54]:
# Fill NaN values in columns 2 to 20 with -1
test_df.iloc[:, 0:19] = test_df.iloc[:, 0:19].fillna(-1)

# Fill NaN values in columns 23 to 26 with 0
test_df.iloc[:, 19:28] = test_df.iloc[:, 19:28].fillna(0)

In [55]:
test_df

,min_value,max_value,mean_value,std_dev,unique_count,special_values,average_length,min_length,max_length,all_caps,...,special_chars,min_date,max_date,date_range,year_counts,month_counts,min_month,min_year,max_month,max_year
0,-1,-1,-1,-1,-1,-1,20.0,20,20,0,...,30,0,0,0,0,0,0.0,0.0,0.0,0.0
